In [28]:
%matplotlib inline
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load the images in gray scale
img1 = cv2.imread('images/scene_with_candy.jpg', 0)
img2 = cv2.imread('images/scene_with_candy_2.jpg', 0)

# Detect the SIFT key points and compute the descriptors for the two images
sift = cv2.xfeatures2d.SIFT_create()
keyPoints1, descriptors1 = sift.detectAndCompute(img1, None)
keyPoints2, descriptors2 = sift.detectAndCompute(img2, None)

# Create brute-force matcher object
bf = cv2.BFMatcher()

# Match the descriptors
matches = bf.knnMatch(descriptors1, descriptors2, k=2)

# Select the good matches using the ratio test
goodMatches = []

for m, n in matches:
    if m.distance < 0.7 * n.distance:
        goodMatches.append(m)

# Apply the homography transformation if we have enough good matches 
MIN_MATCH_COUNT = 10

if len(goodMatches) > MIN_MATCH_COUNT:
    # Get the good key points positions
    sourcePoints = np.float32([ keyPoints1[m.queryIdx].pt for m in goodMatches ]).reshape(-1, 1, 2)
    destinationPoints = np.float32([ keyPoints2[m.trainIdx].pt for m in goodMatches ]).reshape(-1, 1, 2)
    
    # Obtain the homography matrix
    M, mask = cv2.findHomography(sourcePoints, destinationPoints, method=cv2.RANSAC, ransacReprojThreshold=5.0)
    matchesMask = mask.ravel().tolist()
    
    # Apply the perspective transformation to the source image corners
    h, w = img1.shape
    corners = np.float32([ [0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0] ]).reshape(-1, 1, 2)
    transformedCorners = cv2.perspectiveTransform(corners, M)
    
    # Draw a polygon on the second image joining the transformed corners
    img2 = cv2.polylines(img2, [np.int32(transformedCorners)], True, (255, 255, 255), 2, cv2.LINE_AA)
else:
    print("Not enough matches are found - %d/%d" % (len(goodMatches), MIN_MATCH_COUNT))
    matchesMask = None

# Draw the matches
drawParameters = dict(matchColor=(0, 255, 0), singlePointColor=None, matchesMask=matchesMask, flags=2)
result = cv2.drawMatches(img1, keyPoints1, img2, keyPoints2, goodMatches, None, **drawParameters)

# Display the results
#plt.axis("off")
#plt.imshow(result, extent=[0,400,0,1], aspect='auto')
#plt.show()

# Display the results
cv2.imshow('Homography', result)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [54]:
%matplotlib inline
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load the images in gray scale
img1 = cv2.imread('images/scene_with_candy.jpg', 0)
img2 = cv2.imread('images/scene_with_candy_2.jpg', 0)

saliencyDetector = cv2.saliency.StaticSaliencySpectralResidual_create()
img1Values = saliencyDetector.computeSaliency(img1, None)
img2Values = saliencyDetector.computeSaliency(img2, None)

img1Values[1].shape
result = (img1Values[1] > 0.16) * img1Values[1]

array([[ 0.16611283,  0.16611283,  0.16611283, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.16611283,  0.16611283,  0.16611283, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.16611283,  0.16611283,  0.16611283, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)